In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
import chardet

#open data , 영화리뷰데이터 
with open("imdb_master.csv", 'rb') as f:
    result = chardet.detect(f.read()) 

train = pd.read_csv("imdb_master.csv", encoding=result['encoding'])
train.head()


In [ ]:
train = train.drop(train.columns[[0, 1, 4]], axis=1)

In [ ]:
train.label = [(l!='neg')*1 for l in train.label]

In [ ]:
train.head() 

In [ ]:
d = train 
d.columns = ['user_review' , 'positive']
d.head() 

In [ ]:
# 트레이닝셋과 테스트셋으로 나눈다. 
split = 0.7
d_train = d[:int(split*len(d))]
d_test = d[int((1-split)*len(d)):]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(d_train.user_review)
features

In [ ]:
len(vectorizer.get_feature_names()) , vectorizer.get_feature_names()

In [ ]:
i = 30000
j = 100
words = vectorizer.get_feature_names()[i:i+30]
pd.DataFrame(features[j:j+10,i:i+30].todense(), columns=words)

In [ ]:
words

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB()
model1.fit(features, d_train.positive)

pred1 = model1.predict_proba(vectorizer.transform(d_test.user_review))
pred1 , pred1.shape

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, roc_curve
def performance(y_true, pred, color="g", ann=True):
    acc = accuracy_score(y_true, pred[:,1] > 0.5)
    auc = roc_auc_score(y_true, pred[:,1])
    fpr, tpr, thr = roc_curve(y_true, pred[:,1])
    plot(fpr, tpr, color, linewidth="3")
    xlabel("False positive rate")
    ylabel("True positive rate")
    if ann:
        annotate("Acc: %0.2f" % acc, (0.1,0.8), size=14)
        annotate("AUC: %0.2f" % auc, (0.1,0.7), size=14)

In [ ]:
performance(d_test.positive, pred1)